# Mapping classes abcd and abaCDf are conjugate.

### Modules

In [1]:
import sympy as sp
sp.init_printing(use_latex='mathjax')

# 1. Conjugacy on the symplectic representation

## 1.1. Symplectic representation

### Sp_repr class and functions

In [2]:
class Sp_repr:
    #--- 2x2 ---
    I = sp.Identity(2)
    O = sp.ZeroMatrix(2,2)
    J, evJ = sp.MatrixSymbol('J',2,2), sp.Matrix([[0,1],[-1,0]])
    #---
    L, evL = sp.MatrixSymbol('L',2,2), sp.Matrix([[1,0],[1,1]])
    K, evK = sp.MatrixSymbol('K',2,2), sp.Matrix([[0,-1],[0,0]])
    #--- 4x4 ---
    J4_bm = sp.BlockMatrix([[J, O], [O, J]])
    J4 = J4_bm.subs([(J, evJ)]).as_explicit()
    #---
    CHR_2_MTX = {'a': sp.BlockMatrix([[L.inv(), O], [O, I]]),
                 'b': sp.BlockMatrix([[L.transpose(), K], [K, L.transpose()]]),
                 'c': sp.BlockMatrix([[I, O], [O, L.inv()]]),
                 'd': sp.BlockMatrix([[I, O], [O, L.transpose()]]),
                 'f': sp.BlockMatrix([[L.transpose(), O], [O,I]])}
    #---
    
    def __init__(self, l:str):
        self.loop = l
        self.block_matrix = self.CHR_2_MTX.get(l) if l.islower() else self.CHR_2_MTX.get(l.lower()).inv()
        self.matrix = self.block_matrix.subs([
            (self.L, self.evL), (self.K, self.evK) #, (self.tl, self.evtl), (self.br, self.evbr)
        ]).as_explicit()

In [3]:
def is_Sp(M):
    J = Sp_repr.J4
    return M.transpose()*J*M == J
    
list(map(lambda l: is_Sp(Sp_repr(l).matrix), ['a','b','c','d','f', 'A'])) 

[True, True, True, True, True, True]

### The symplectic images of Abcd and abcD

In [4]:
a,b,c,d,f = tuple(map(lambda l: Sp_repr(l).matrix, ['a','b','c','d','f']))
J4 = Sp_repr.J4

a,b,c,d,f, J4 

(Matrix([
 [ 1, 0, 0, 0],
 [-1, 1, 0, 0],
 [ 0, 0, 1, 0],
 [ 0, 0, 0, 1]]),
 Matrix([
 [1,  1, 0, -1],
 [0,  1, 0,  0],
 [0, -1, 1,  1],
 [0,  0, 0,  1]]),
 Matrix([
 [1, 0,  0, 0],
 [0, 1,  0, 0],
 [0, 0,  1, 0],
 [0, 0, -1, 1]]),
 Matrix([
 [1, 0, 0, 0],
 [0, 1, 0, 0],
 [0, 0, 1, 1],
 [0, 0, 0, 1]]),
 Matrix([
 [1, 1, 0, 0],
 [0, 1, 0, 0],
 [0, 0, 1, 0],
 [0, 0, 0, 1]]),
 Matrix([
 [ 0, 1,  0, 0],
 [-1, 0,  0, 0],
 [ 0, 0,  0, 1],
 [ 0, 0, -1, 0]]))

In [5]:
A, B, C, D, F = (k.inv() for k in [a,b,c,d,f])
A,B,C,D,F

(Matrix([
 [1, 0, 0, 0],
 [1, 1, 0, 0],
 [0, 0, 1, 0],
 [0, 0, 0, 1]]),
 Matrix([
 [1, -1, 0,  1],
 [0,  1, 0,  0],
 [0,  1, 1, -1],
 [0,  0, 0,  1]]),
 Matrix([
 [1, 0, 0, 0],
 [0, 1, 0, 0],
 [0, 0, 1, 0],
 [0, 0, 1, 1]]),
 Matrix([
 [1, 0, 0,  0],
 [0, 1, 0,  0],
 [0, 0, 1, -1],
 [0, 0, 0,  1]]),
 Matrix([
 [1, -1, 0, 0],
 [0,  1, 0, 0],
 [0,  0, 1, 0],
 [0,  0, 0, 1]]))

In [6]:
M = sp.block_collapse(a*b*c*d)
display(M)
N = sp.block_collapse(a*b*a*C*D*f)
display(N)

list(map(is_Sp, [M, N]))

Matrix([
[ 1,  1,  1, 0],
[-1,  0, -1, 0],
[ 0, -1,  0, 1],
[ 0,  0, -1, 0]])

Matrix([
[ 0,  1, -1,  0],
[-1, -1,  1,  0],
[ 1,  0,  2, -1],
[ 0,  0,  1,  0]])

[True, True]

### The charactoristic polynomial of M (and N)

In [7]:
cp_M = M.charpoly().as_expr().factor()
display(cp_M)

cp_N = N.charpoly().as_expr()
print(f"cp_M == cp_N ?   --> {cp_M==cp_N}")

lambda**4 - lambda**3 + lambda**2 - lambda + 1

cp_M == cp_N ?   --> True


## 1.2. Rational canonical form

### The rational form of M and N

In [8]:
Cpn = sp.matrices.expressions.CompanionMatrix(M.charpoly()).as_explicit()
display(Cpn)
print(f"is_Sp? --> {is_Sp(Cpn)}")

Matrix([
[0, 0, 0, -1],
[1, 0, 0,  1],
[0, 1, 0, -1],
[0, 0, 1,  1]])

is_Sp? --> False


In [9]:
def transition_matrix(M,v):
    if type(v) == list:
        v = sp.Matrix(4,1,v)
    vecs = [(M**k)*v for k in range(5)]
    return sp.Matrix(4,4,lambda i,j: vecs[j][i])
def tm(M,v):
    return transition_matrix(M,v)

In [10]:
from random import randint

v = [randint(-10,10) for _ in range(4)]
P = tm(M, v)

v, P, (P.inv())*M*P

([-4, 2, -9, -9],
 Matrix([
 [-4, -11, -9,   9],
 [ 2,  13, 22,  13],
 [-9, -11, -4, -11],
 [-9,   9, 11,   4]]),
 Matrix([
 [0, 0, 0, -1],
 [1, 0, 0,  1],
 [0, 1, 0, -1],
 [0, 0, 1,  1]]))

## 1.3. Base symplectic matrix $S_0$

In [11]:
def conjugation_matrix(v, w):
    P, Q = tm(M,v), tm(N,w)
    return P*(Q.inv())

In [12]:
v0, w0 = [0,-1,-1,1], [0,1,0,1]
S0 = conjugation_matrix(v0,w0)
display(S0)
print(f"{(N == S0.inv()*M*S0)=}, {is_Sp(S0)=}")

Matrix([
[-3,  0, -1,  0],
[ 1, -1,  1,  0],
[ 1, -2,  1,  1],
[ 4,  2,  1, -1]])

(N == S0.inv()*M*S0)=True, is_Sp(S0)=False


# Ideal for fibered knots

In [18]:
display(M)

Matrix([
[ 1,  1,  1, 0],
[-1,  0, -1, 0],
[ 0, -1,  0, 1],
[ 0,  0, -1, 0]])

In [25]:
P = sp.Matrix([[0,1,2,0],[-1,-2,-2,2],[0,0,-1,0],[0,0,0,1]])
display(P, P^(-1), is_Sp(P))

Matrix([
[ 0,  1,  2, 0],
[-1, -2, -2, 2],
[ 0,  0, -1, 0],
[ 0,  0,  0, 1]])

Matrix([
[-2, -1, -2, 2],
[ 1,  0,  2, 0],
[ 0,  0, -1, 0],
[ 0,  0,  0, 1]])

False

In [28]:
N == P*M*P^(-1)

True